In [1]:
# data

text = None
data_file = "input.txt"
with open(data_file, "r") as f:
    text = f.read()
print("Estimated tokens:", len(text))


Estimated tokens: 1115393


In [2]:
# Tokenizer

chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)
print("Unique characters:", ''.join(chars), end="\n\n")

i2char = { x:chars[x] for x in range(len(chars)) }
char2i = { chars[x]:x for x in range(len(chars)) }


encode = lambda string_ : [ char2i[s] for s in string_  ]
decode = lambda tokens : ''.join([ i2char[t] for t in tokens  ])

test_str = 'hi there'
print(f"encoding '{test_str}' gives {encode(test_str)}")


Unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

encoding 'hi there' gives [46, 47, 1, 58, 46, 43, 56, 43]


In [3]:
# train test split
import torch

split_ratio = .9
size = len(text)
n = int( size * split_ratio)

encoded_text = torch.tensor(encode(text))
train_dataset = encoded_text[:n]
val_dataset = encoded_text[n:]

print(f"train : val split ====> {len(train_dataset)/size :.2f}:{len(val_dataset)/size :.2f}")



train : val split ====> 0.90:0.10


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split, batch_size , block_size ):
    # generate a small batch of data of inputs x and targets y

    assert split in ['train', 'val']
    data = train_dataset if split == 'train' else val_dataset

    ix = torch.randint(low = 0, high = len(data) - batch_size, size=(batch_size,) )
    x = torch.stack([ data[i: i+block_size] for i in ix])
    y = torch.stack([ data[i+1: i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)
    return x, y


In [ ]:
import torch.nn as nn

torch.manual_seed(1337)
batch_size = 4
block_size = 8


class SimpleBigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 

    def forward(self, idx, targets=None):
        # idx == (B, T)
        logits = self.token_embedding_table(idx) # B,T,C === Batch, Time, Channels === Rows, Columns, Channels

        if targets == None:
            loss = None
        else:
            B,T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = torch.nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx,  max_new_tokens: int):
        # idx == (B, T)

        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # focus on last time step (col) ===> becomes (B, C)
            probs = torch.nn.functional.softmax(logits, dim=-1) # apply on C
            idx_next = torch.multinomial(probs, num_samples=1) # # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx



In [5]:
# training

# print(x)
# print(y)

model = SimpleBigramLanguageModel(vocab_size).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
max_iters = 3000
eval_interval = 300

model.train()

for epoch in range(max_iters):
    x, y = get_batch('train')

    if epoch % eval_interval == 0:
        model.eval()
        x_train, y_train = get_batch('train')
        x_val, y_val = get_batch('val')
        
        _, train_loss = model(x_train, y_train)
        _, val_loss = model(x_val, y_val)

        print(f"epoch {epoch}: train loss {train_loss :.4f}, val loss {val_loss:.4f} ")
        model.train()

    logits, loss = model(x, y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



epoch 0: train loss 4.6291, val loss 4.6232 
epoch 300: train loss 3.1551, val loss 3.0098 
epoch 600: train loss 3.0026, val loss 2.4398 
epoch 900: train loss 3.0212, val loss 2.8308 
epoch 1200: train loss 2.3367, val loss 2.3471 
epoch 1500: train loss 2.3850, val loss 2.5813 
epoch 1800: train loss 2.5350, val loss 2.5518 
epoch 2100: train loss 2.3550, val loss 2.3588 
epoch 2400: train loss 2.6374, val loss 2.4671 
epoch 2700: train loss 2.2695, val loss 2.1183 


In [6]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


JOLI ou walur paly houmy rde t die;
CHFigovigeeat ueju ly, cowiccond d hand wouce ds nlowisheno, Inds s tasthid thy, thaimond's k; Oun'ssearethe fosonsigrisaples, br, tronshe lveald Wesourar
gis?atr, l be-d,
ENGyoy Em n.
chodn mfe me! fodapas. habyomamalwous sushaglowiress; yFlwelo, oowityo hy, e was t ms,
Thmind in, d y s imatheaccarcano thidsto boinerougre; E:
Burs higrind blos.

Y:

DYoupe cofe'diea! helalpou,
&'sV&'sprals.


Cay'le taie OHese
BELond ale g.
BUzencust s t rd t
FhRI y thend:
SC


In [7]:
# The whole token/ node communication explanation. From averaging to attention
# Attention is a communication mechanism

torch.manual_seed(1337)

B,T,C = 4, 8, 2
x = torch.randn(B, T,C)

print('-----')
# version 1 - averaging past context using for loops
z_1 = torch.zeros(B,T, C)
for b in range(B):
    for t in range(T):
        x_prev = x[b, :t+1]
        z_1[b, t] = torch.mean(x_prev, 0)

# print(x[0])
# print(z_1[0])

# version 2 matrix multiply
print('-----')
a = torch.ones(3,3 )

print(a.shape)
print(a)
a = torch.tril(a)
a = a / torch.sum(a, 1, keepdim=True)
print("a\n", a)

b = torch.randint(0, 10, (3,2)).float()
print("b\n", b)

c = a@b
print("c\n", c)
# z = torch.zeros(B, T, C)

print('-----')

# version 3 : matrix multiply average using softmax

a = torch.ones(3,3 )
a = torch.tril(a)
a = a.masked_fill(a==0, float('-inf'))
a = torch.softmax(a,  -1)
b = torch.randint(0, 10, (3,2)).float()
c = a@b

print("a\n", a)
print("b\n", b)
print("c\n", c)

print('-----')



-----
-----
torch.Size([3, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
a
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b
 tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
c
 tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])
-----
a
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b
 tensor([[7., 4.],
        [5., 0.],
        [5., 3.]])
c
 tensor([[7.0000, 4.0000],
        [6.0000, 2.0000],
        [5.6667, 2.3333]])
-----


In [37]:
import torch.nn as nn

# gpt hyperparams

batch_size = 16
block_size = 128
learning_rate = 3e-4
n_embd = 192
n_head = 6
n_layer = 6 # decoder layers
dropout = 0.2


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()

        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        wei = (q @ k.transpose(-2,-1)) * k.shape[-1] ** -0.5
        wei = torch.nn.functional.softmax( wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) , dim=-1) # this step removed in encoder type transformer
        wei = self.dropout(wei)
        out = wei @ v
        return out

class MultiheadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()

        self.heads = nn.ModuleList( [ Head(head_size) for _ in range(num_heads) ] )
        self.proj = nn.Linear(head_size * num_heads , n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([ h(x) for h in self.heads ], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()

        head_size = n_embd// n_head
        self.mha = MultiheadAttention(n_head, head_size)
        self.ffw = FeedForward(n_embd)
        
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        out = self.mha(self.ln1(x))
        out = self.ffw(self.ln2(out))
        return out

    
class GPT(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        self.blocks = nn.Sequential( *[ Block(n_embd, n_head) for _ in range(n_layer) ] )

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)


    # copied pasted this part, didnt even type it out
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, idx, targets = None):
        # print("any nan in idx:", torch.isnan(idx).any())

        B, T = idx.shape

        token_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        # print(token_emb.shape)
        # print(pos_emb.shape)

        # print("tok " ,torch.isnan(token_emb).any())
        # print("pos ", torch.isnan(pos_emb).any())

        x = token_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = torch.nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # focus on last time step (col) ===> becomes (B, C)
            probs = torch.nn.functional.softmax(logits, dim=-1) # apply on C
            idx_next = torch.multinomial(probs, num_samples=1) # # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    



In [40]:
max_iters = 2000
eval_interval = 100


model = GPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        x_train, y_train = get_batch('train', batch_size = batch_size, block_size = block_size)
        x_val, y_val = get_batch('val', batch_size = batch_size, block_size = block_size)
        
        _, train_loss = model(x_train, y_train)
        _, val_loss = model(x_val, y_val)

        print(f"epoch {iter}: train loss {train_loss :.4f}, val loss {val_loss:.4f} ")

    # sample a batch of data
    xb, yb = get_batch('train', batch_size = batch_size, block_size = block_size)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()






2.715713 M parameters
epoch 0: train loss 4.2309, val loss 4.2265 
epoch 100: train loss 3.3988, val loss 3.4534 
epoch 200: train loss 3.3905, val loss 3.3924 
epoch 300: train loss 3.3539, val loss 3.3629 
epoch 400: train loss 3.3060, val loss 3.3286 


RuntimeError: stack expects each tensor to be equal size, but got [128] at entry 0 and [37] at entry 8

In [41]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


h e ot  nuwo  thwtcCda e Ooiinht,tr ta:-Su
  nlelPpbcg
tohtrrhb Nnn Likeeattflr tseRet;ohiuminL ita

 hs nTsy An  tehrttadA  resshhvh llree  Oh A e noa  si'd sBnnla AtewrafuaT rtar
ePI  io Rio
itnu hnuu vsCha
ys aIr NE ei wdnilu omhantmdL lcRshre
eo onmtas
Ntw lae iamaut  hhTe ,irhryllIe
ioNsagoA wi t Yroaimno, etaeSt.anhbM.bs,l LsesbLt uheucolgdaw rUtiTnlM nvn
 
uaficewwwuut
 seMrbcsg  ieiw ohs
y ot flno aueedhatm os

arro akeodaa nieti ,o
dlM
rf,n Yr,rsgh yntohfvvceoiObnyt:idttennyiofea  al
 q
